In [2]:
import numpy as np

#convert right ascension from hms to dec
def hms2dec(hms1,hms2,hms3):
    dec = (hms1 * 15) + ((hms2 / 60) *15) + ((hms3 /(60**2))*15)
    return dec

#convert declination from dms to dec
def dms2dec(dms1,dms2,dms3):
    if dms1 < 0:
        a = -1
    else:
        a = 1
    dec = dms1 + ((a * dms2 / 60)) + (a*(dms3 /(60**2)))
    return dec

#calculate angular distance between objects
def angular_dist(ra1,dec1,ra2,dec2):
    r1 = np.radians(ra1)
    r2 = np.radians(ra2)
    d1 = np.radians(dec1)
    d2 = np.radians(dec2)
    
    a = np.sin((np.abs(d1-d2))/2)**2
    b = np.cos(d1)*np.cos(d2)*np.sin(np.abs(r1 - r2)/2)**2
    d = 2*np.arcsin(np.sqrt(a+b))
    d_d = np.degrees(d)
    return d_d

#write a function that imports bss and super catalogues
def import_bss():
    res = []
    data = np.loadtxt('bss.dat', usecols=range(1, 7))
    for r, row in enumerate(data, 1):
        res.append((r, hms2dec(row[0], row[1], row[2]), dms2dec(row[3], row[4], row[5])))
    return res

def import_super():
    data = np.loadtxt('super.csv', delimiter=',', skiprows=1, usecols=(0, 1))
    res = []
    for r, row in enumerate(data, 1):
        res.append((r, row[0], row[1]))
    return res

#find closest thing to a target by looping through each object in a catalogue, calculating its distance from the target
#and then recording its distance as min_dist if it has the lowest distance yet
def find_closest(cat, ra, dec):
    min_dist = np.inf
    min_id = None
    for id1, ra1, dec1 in cat:
        dist = angular_dist(ra1, dec1, ra, dec)
        if dist < min_dist:
            min_id = id1
            min_dist = dist
    
    return min_id, min_dist

#write the naive cross matching algorithm
def crossmatch(cat1, cat2, max_radius):
    cat1 = import_bss()
    cat2 = import_super()
    matches = []
    no_matches = []
    #loop through each object in cat1 and determine its distance to each object in cat2 using nested loops
    for id1, ra1, dec1 in cat1:
        closest_dist = np.inf
        closest_id2 = None
        for id2, ra2, dec2 in cat2:
            dist = angular_dist(ra1, dec1, ra2, dec2)
            if dist < closest_dist:
                closest_id2 = id2
                closest_dist = dist
        
        # ignore if outside max radius
        if closest_dist > max_radius:
            no_matches.append(id1)
        #keep and append it to the matches list
        else:
            matches.append((id1, closest_id2, closest_dist))

    return matches

crossmatch('bss.dat','super.csv',10)

[(1, 1, 5.311246494367285),
 (2, 3, 2.7573607427578017),
 (3, 3, 1.497926848883644)]

In [ ]:
#optimise the crossmatching algorithm by converting the catalogues to radians right at the start instead of the individual coord
# Write your crossmatch function here.
import numpy as np
import time

def angular_dist(r1, d1, r2, d2):
    a = np.sin(np.abs(d1 - d2)/2)**2
    b = np.cos(d1)*np.cos(d2)*np.sin(np.abs(r1 - r2)/2)**2
    return 2*np.arcsin(np.sqrt(a + b))

def crossmatch(cat1, cat2, max_radius):
    start = time.perf_counter()
    max_radius = np.radians(max_radius)
  
    matches = []
    no_matches = []

    #convert whole thing to radians
    cat1 = np.radians(cat1)
    cat2 = np.radians(cat2)

    for id1, (ra1, dec1) in enumerate(cat1):
        min_dist = np.inf
        min_id2 = None
        for id2, (ra2, dec2) in enumerate(cat2):
            #angular dist doesn't convert to radians anymore
            dist = angular_dist(ra1, dec1, ra2, dec2)
            if dist < min_dist:
                min_id2 = id2
                min_dist = dist
        
        if min_dist > max_radius:
            no_matches.append(id1)
        else:
            matches.append((id1, min_id2, np.degrees(min_dist)))
    
    time_taken = time.perf_counter() - start
    return matches, no_matches, time_taken

#use numpy arrays instead of lists to speed it up
def angular_dist(r1, d1, r2, d2):
    a = np.sin(np.abs(d1 - d2)/2)**2
    b = np.cos(d1)*np.cos(d2)*np.sin(np.abs(r1 - r2)/2)**2
    return 2*np.arcsin(np.sqrt(a + b))

def crossmatch(cat1, cat2, max_radius):
    start = time.perf_counter()
    max_radius = np.radians(max_radius)
  
    matches = []
    no_matches = []

  # Convert coordinates to radians
    cat1 = np.radians(cat1)
    cat2 = np.radians(cat2)
    ra2s = cat2[:,0]
    dec2s = cat2[:,1]

    for id1, (ra1, dec1) in enumerate(cat1):
        dists = angular_dist(ra1, dec1, ra2s, dec2s)
        min_id = np.argmin(dists)
        min_dist = dists[min_id]
        if min_dist > max_radius:
            no_matches.append(id1)
        else:
            matches.append((id1, min_id, np.degrees(min_dist)))
    
    time_taken = time.perf_counter() - start
    return matches, no_matches, time_taken

# Sort cat2 by declination and then work upwards, reducing the time it takes
import numpy as np
import time

def angular_dist(r1, d1, r2, d2):
    a = np.sin(np.abs(d1 - d2)/2)**2
    b = np.cos(d1)*np.cos(d2)*np.sin(np.abs(r1 - r2)/2)**2
    return 2*np.arcsin(np.sqrt(a + b))

def crossmatch(cat1, cat2, max_radius):
    start = time.perf_counter()
    max_radius = np.radians(max_radius)
  
    matches = []
    no_matches = []

    # Convert coordinates to radians
    cat1 = np.radians(cat1)
    cat2 = np.radians(cat2)
    order = np.argsort(cat2[:,1])
    cat2_ordered = cat2[order]
  
    for id1, (ra1, dec1) in enumerate(cat1):
        min_dist = np.inf
        min_id2 = None
        max_dec = dec1 + max_radius
        for id2, (ra2, dec2) in enumerate(cat2_ordered):
            if dec2 > max_dec:
                break
        
            dist = angular_dist(ra1, dec1, ra2, dec2)
            if dist < min_dist:
                min_id2 = order[id2]
                min_dist = dist

        if min_dist > max_radius:
            no_matches.append(id1)
        else:
            matches.append((id1, min_id2, np.degrees(min_dist)))
    
    time_taken = time.perf_counter() - start
    return matches, no_matches, time_taken